### Task 105: Simulate Non-rebasing token being deposited and traded against rebasing token
* Simulate CPT trading with a non-rebasing toke paired with a rebasing token.

In [1]:
import os
import copy
import numpy as np
import time
import matplotlib.pyplot as plt
cwd =  os.getcwd().replace("notebooks/test","")
os.chdir(cwd)

from python.dev.event.state.series import StateSeries
from python.dev.event.state import State
from python.dev.event import Deposit
from python.dev.event import Withdraw
from python.dev.token.trade import Convert
from python.dev.cpt import Liquidity
from python.dev.cpt import PriceCurve
from python.dev.cpt import ConstantProductTrade
from python.dev.plot import PlotPriceCurve
from python.dev.token import RebaseToken
from python.dev.token import NonRebaseToken
from python.dev.math.model import TimeDeltaModel
from python.dev.math.model import TokenDeltaModel
from python.dev.math.model import EventSelectionModel

In [2]:
time0 = time.time()

* Initialize Rebasing and Non-rebasing tokens along with liquidity pool

In [3]:
# Initialize Rebasing/NonRebasing Tokens
APY = 0.1

TYPE_DEPOSIT_TOKEN = 'DEPOSIT'
TYPE_COLLATERAL_TOKEN = 'COLLATERAL'

time_delta = TimeDeltaModel().apply()
depositT1 = 10000  

T1 = NonRebaseToken(TYPE_DEPOSIT_TOKEN)
t1Address = T1.gen_address()
T1.init_token(time0, t1Address)
T1.add_event(Deposit(APY, depositT1, time_delta, t1Address))

time_delta = TimeDeltaModel().apply()
depositTCT1 = 10000 

TCT1 = RebaseToken(TYPE_COLLATERAL_TOKEN)
tctAddress = TCT1.gen_address()
TCT1.init_token(time0, tctAddress)
TCT1.add_event(Deposit(APY, depositTCT1, time_delta, tctAddress))

x = T1.get_balance_deposits()
y = TCT1.get_balance_deposits()

print('[Initial] T1: {:.2f} TCT1: {:.2f}\n'.format(x,y))

# Initialize Liquidity Pool
liq = Liquidity(x,y)
L = liq.calc(); p = np.random.normal(5,0.1,1)[0]
cpt = ConstantProductTrade(L)
pCurve = PriceCurve(L)

[Initial] T1: 10000.00 TCT1: 10000.00



* The deposit event that includes the amount of non-rebasing token will expand the supply of the rebasing token 1:1

In [7]:
token_delta = TokenDeltaModel(1,100).apply() 
time_delta = TimeDeltaModel().apply() 
T1.add_event(Deposit(APY, token_delta, time_delta, t1Address))
TCT1.add_event(Deposit(APY, token_delta, time_delta, tctAddress)) 

TCT1_yield = TCT1.get_state_series(tctAddress).get_last_state().get_yield()

x = T1.get_balance_deposits()
y = TCT1.get_balance_deposits()

liq.delta_y(token_delta+TCT1_yield)
liq.delta_x(token_delta) 
L = liq.calc()

print('[Deposits] T1: {:.5f} TCT1: {:.5f} L: {:.5f}'.format(x,y,L))


[Deposits] T1: 9996.74871 TCT1: 10648.89247 L: 10317.66941


* The subsequent trade must be for the same amount being deposited.
* The amount of rebasing token purchase must be reflected as having left CPT liquidity.

In [9]:
T1.add_event(Withdraw(APY, token_delta, 0, t1Address))
TCT1.add_event(Deposit(APY, token_delta, 0, tctAddress)) 

x = T1.get_balance_deposits()
y = TCT1.get_balance_deposits()

liq.delta_y(token_delta)
liq.delta_x(-token_delta) 

L = liq.calc()

print('[Swap] T1: {:.5f} TCT1: {:.5f} L: {:.5f}'.format(x,y,L))


[Swap] T1: 9681.61894 TCT1: 10964.02224 L: 10302.88724


* The amount of rebasing token purchase must be reflected as having left CPT liquidity.

In [ ]:
# Plot and execute N trades
percent = 0.05
N = 250

plotCurve = PlotPriceCurve(L, cpt, pCurve)
plotCurve.apply(p, None)

for k in range(N):
    # Generate random delta denomenated in Native token, 
    # rebalance respective accounts and convert to T1
    delta_x = TokenDeltaModel(1,100).apply() 
    t_delta = TimeDeltaModel().apply()

    select = EventSelectionModel().bi_select(0.5)
    if(select == EventSelectionModel().FIRST): 
        T1.add_event(Withdraw(APY, delta_x, t_delta, t1Address))
        TDT.add_event(Deposit(APY, delta_x, t_delta, tdtAddress))               
    else:            
        T1.add_event(Deposit(APY, delta_x, t_delta, t1Address))
        TDT.add_event(Withdraw(APY, delta_x, t_delta, tdtAddress))         
    
    TDT_yield = TDT.get_state_series(tdtAddress).get_state(k).get_yield()
    T1_yield = T1.get_state_series(t1Address).get_state(k).get_yield()
    
    liq.delta_y(TDT_yield)
    
    L = liq.swap(delta_x); p = np.random.normal(5,0.1,1)[0]
    if (k % int(N/(100*percent)) == 0):
        plotCurve.apply_next(p, L, '({}) L = {}, p = {:.3f}'.format(k, int(L),p))
        plotCurve.plot_trade(p, delta_x, 'r--')
        T1_prin = T1.get_state_series(t1Address).get_state(k).get_principle()
        T1_balance = T1.get_state_series(t1Address).get_state(k).get_balance()

        TDT_prin = TDT.get_state_series(tdtAddress).get_state(k).get_principle()
        TDT_balance = TDT.get_state_series(tdtAddress).get_state(k).get_balance()
            
        print('[Trade {}] T1: {:.5f} TDT: {:.5f} L: {:.5f}'.format(k, x,y,L))
        print('[Trade {}] T1_prin: {:.5f} T1_balance: {:.5f}'.format(k, T1_prin,T1_balance))
        print('[Trade {}] TDT_prin: {:.5f} TDT_balance: {:.5f}\n'.format(k, TDT_prin,TDT_balance))
      
    x = T1.get_balance_deposits()
    y = TDT.get_balance_deposits()



In [ ]:
# Analyze Token balances

T1_states = T1.get_state_series(t1Address)
principle = T1_states.get_principle() 
balance = T1_states.get_balance() 
tSamp = T1_states.get_tstamp()  

fig, (T1_ax, TDT_ax) = plt.subplots(nrows=2, sharex=True, sharey=False, figsize=(12, 10))

T1_ax.plot(tSamp[1:], balance[1:], label='balance') 
T1_ax.plot(tSamp[1:], principle[1:], label='principle') 
T1_ax.set_title("T1 Balance (in native token)", fontsize=20)
T1_ax.set_ylabel("Native token", fontsize=16)
T1_ax.set_xlabel('Time', fontsize=16)
T1_ax.legend()

TDT_states = TDT.get_state_series(tdtAddress)
principle = TDT_states.get_principle() 
balance = TDT_states.get_balance() 
tSamp = TDT_states.get_tstamp() 

TDT_ax.plot(tSamp[1:], balance[1:], label='balance') 
TDT_ax.plot(tSamp[1:], principle[1:], label='principle') 
TDT_ax.set_title("TDT Balance (in native token)", fontsize=20)
TDT_ax.set_ylabel("native token", fontsize=16)
TDT_ax.set_xlabel('Time', fontsize=16)
TDT_ax.legend()
